In [ ]:
from confluent_kafka import Consumer
import json

conf = {
    'bootstrap.servers': 'broker1-kr:9092',
    'group.id': 'taxi_consumer_group',
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(conf)
consumer.subscribe(['yellow_taxi_stream'])

print("Consuming messages from 'yellow_taxi_stream'...")

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print("Consumer error:", msg.error())
            continue

        data = json.loads(msg.value().decode('utf-8'))
        print("Consumed:", data)
except KeyboardInterrupt:
    pass
finally:
    consumer.close()


In [ ]:
from confluent_kafka import Consumer
import json

conf = {
    'bootstrap.servers': 'broker1-kr:9092',
    'group.id': 'taxi_consumer_group',
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(conf)
consumer.subscribe(['yellow_taxi_stats'])

print("Consuming messages from 'yellow_taxi_stream'...")

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print("Consumer error:", msg.error())
            continue

        data = json.loads(msg.value().decode('utf-8'))
        print("Consumed:", data)
except KeyboardInterrupt:
    pass
finally:
    consumer.close()